In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


2025-07-06 09:35:20.894973: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751794521.168008      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751794521.241110      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Paths (Kaggle - attached dataset)
train_folder = '/kaggle/input/dog-breed-identification/train/'
test_folder = '/kaggle/input/dog-breed-identification/test/'
label_file = '/kaggle/input/dog-breed-identification/labels.csv'



In [3]:
df = pd.read_csv(label_file)
breed_to_index = {breed: i for i, breed in enumerate(df['breed'].unique())}
index_to_breed = {i: breed for breed, i in breed_to_index.items()}  # optional


In [4]:
from tensorflow.keras.preprocessing import image

def load_train_data(df, image_folder, img_size=(224, 224)):
    images, labels, ids = [], [], []
    label_map = dict(zip(df['id'], df['breed']))

    for file_name in os.listdir(image_folder):
        if file_name.endswith('.jpg'):
            img_id = file_name.split('.')[0].strip()
            label = label_map.get(img_id)

            if label:
                img_path = os.path.join(image_folder, file_name)
                img = image.load_img(img_path, target_size=img_size)
                img = image.img_to_array(img) / 255.0
                images.append(img)
                labels.append(breed_to_index[label])
                ids.append(img_id)

    X = np.array(images)
    y = to_categorical(np.array(labels), num_classes=len(breed_to_index))
    return X, y, ids


In [5]:
def load_test_data(image_folder, img_size=(224, 224)):
    images, ids = [], []
    for file_name in os.listdir(image_folder):
        if file_name.endswith('.jpg'):
            img_path = os.path.join(image_folder, file_name)
            img = image.load_img(img_path, target_size=img_size)
            img = image.img_to_array(img) / 255.0
            images.append(img)
            ids.append(file_name.split('.')[0])
    return np.array(images), ids


In [6]:
X_train, y_train, train_ids = load_train_data(df, train_folder)
X_test, test_ids = load_test_data(test_folder)

print(f" X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f" X_test: {X_test.shape}, test_ids: {len(test_ids)}")


 X_train: (10222, 224, 224, 3), y_train: (10222, 120)
 X_test: (10357, 224, 224, 3), test_ids: 10357


In [7]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train,
    test_size=0.2,
    random_state=42,
    stratify=y_train.argmax(axis=1)
)


In [9]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=2,
    restore_best_weights=True
)

In [10]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

from tensorflow.keras.applications import MobileNetV2

from tensorflow.keras.regularizers import l2

base_model = MobileNetV2(
    weights=None,
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.load_weights('/kaggle/input/mobilenet_v2_weights_1.0_224_no_top.h5/keras/default/1/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5')
base_model.trainable = False


model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.6),
    Dense(len(breed_to_index), activation='softmax',kernel_regularizer=l2(1e-4))
])


2025-07-06 09:43:52.437603: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [11]:
model.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32, callbacks = [early_stop]
)


Epoch 1/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 266s 1s/step - accuracy: 0.1456 - loss: 4.0126 - val_accuracy: 0.6548 - val_loss: 1.3642
Epoch 2/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 264s 1s/step - accuracy: 0.5096 - loss: 1.7713 - val_accuracy: 0.6998 - val_loss: 1.0569
Epoch 3/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 255s 997ms/step - accuracy: 0.6115 - loss: 1.3757 - val_accuracy: 0.7090 - val_loss: 1.0138
Epoch 4/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 256s 1s/step - accuracy: 0.6522 - loss: 1.2023 - val_accuracy: 0.7208 - val_loss: 0.9771
Epoch 5/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 271s 1s/step - accuracy: 0.6875 - loss: 1.0915 - val_accuracy: 0.7271 - val_loss: 0.9365
Epoch 6/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 291s 1s/step - accuracy: 0.7289 - loss: 0.9262 - val_accuracy: 0.7276 - val_loss: 0.9329
Epoch 7/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 277s 1s/step - accuracy: 0.7315 - loss: 0.9054 - val_accuracy: 0.7291 - val_loss: 0.9526
Epoch 8/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 266s 1s/step - accuracy: 0.7456 - loss: 0.8600 - val_a

In [12]:
model.save('cnn_model.h5')

In [16]:
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np

# Load saved model
model = load_model("cnn_model.h5")
print(" Model loaded successfully!")

# Evaluate on validation set
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f" Validation Accuracy: {val_acc:.4f}")

# Predict on test set
pred_probs = model.predict(X_test)

# Prepare DataFrame with breed probabilities
# Ensure columns are in the same order as used in training
breed_names = list(breed_to_index.keys())  # or sorted(breed_to_index.keys()) if needed
submission_df = pd.DataFrame(pred_probs, columns=breed_names)

# Add 'id' column
submission_df.insert(0, "id", [img_id + ".jpg" for img_id in test_ids])

# Save to CSV
submission_df.to_csv('/kaggle/working/submission_final.csv', index=False)
print(" submission.csv saved successfully!")



 Model loaded successfully!
64/64 ━━━━━━━━━━━━━━━━━━━━ 57s 815ms/step - accuracy: 0.7320 - loss: 0.9032
 Validation Accuracy: 0.7311
324/324 ━━━━━━━━━━━━━━━━━━━━ 268s 823ms/step
 submission.csv saved successfully!
